# Collaborative Filtering

This notebook has implementation of Collaborative filtering by optimizing a single cost function to learn both User features and Product features.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix

#### Initializing User featuers and Product features.

$n_{u}$ is the number of unique users.  
$n_{m}$ is the number of unique products.  
$n_{f}$ is the size of the feature vector that we want to use for both the user and the product.  
    
We use normal distribution for both the user and product features with mean 0 and standard deviation 0.1

In [ ]:
def initialize_features(n_u, n_m, n_f):
    # mean and standard deviation
    mu, sigma = 0, 0.1 
    user_features = np.random.normal(mu, sigma, (n_u, n_f))
    prod_features = np.random.normal(mu, sigma, (n_m, n_f))
    return user_features, prod_features

#### Predicting Ratings for given users and products

Since, taking a dot product of user_features and product_features will result in a $n_{u} x n_{m}$ matrix which runs out of memory. We use [scipy sparse matrix implementation](https://docs.scipy.org/doc/scipy/reference/sparse.html) library to avoid memory error.

The inputs to predict_ratings method are $user\_features, product\_features$ and $users, products$. The $users, products$ are list of users who rated respective products. This can be used to index user_features and product_features to ensure that the result matrix is sparse


In [ ]:
def predict_ratings(user_features, prod_features, users, products):
    predicted_ratings = np.sum(np.multiply(user_features[users,:], prod_features[products,:]), axis=1)
    return csr_matrix((predicted_ratings, (users, products)))

#### Computing cost 

Given the features and predictions of ratings based on the features.

Cost function <b>J</b> is given by  
$$ J (x^{(1)},...,x^{(n_{m})}, c^{(1)},...,c^{(n_{u})}) = \frac{1}{2}\sum_{(i,j):r(i,j)=1} ((c^{(j)})^{T}x^{(i)} - y^{(i,j)})  +  \frac{\lambda}{2}\sum_{i=1}^{n_{m}}\sum_{k=1}^{n}(x_{k}^{(i)})^{2}  +  \frac{\lambda}{2}\sum_{i=1}^{n_{m}}\sum_{k=1}^{n}(c_{k}^{(i)})^{2} $$

Where,  
$ x^{(i)} $ is the $i^{th}$ product's feature vector. It is of dimension (n_f, 1)  
$ c^{(i)} $ is the $i^{th}$ user's feature vector. It is of dimension (n_f, 1)  
$ x_{k}^{(i)} $ is the $k^{th}$ feature of $i^{th}$ product's feature vector.  
$ c_{k}^{(i)} $ is the $k^{th}$ feature of $i^{th}$ user's feature vector.  
$ (c^{(j)})^{T}x^{(i)} $ is the predicted rating given by $j^{th}$ user to $i^{th}$ product  
$ y^{(i,j)} $ is the actual rating given by $j^{th}$ user to $i^{th}$ product  
$ \lambda $ is the regularization factor.  
$ r^{(i,j)} = 1$ if $i^{th}$ product is rated by $j^{th}$ user otherwise it is $0$. This is not used in our computation as it is taken care in the predict_ratings method by using indexing based on $user, products$ columns

In [ ]:
def compute_cost(user_features, prod_features, prediction_sparse_matrix, ratings_sparse_matrix, lambd=0.1):
    user_regularization = (lambd/2)*np.sum(np.sum(np.square(user_features)))
    song_regularization = (lambd/2)*np.sum(np.sum(np.square(song_features)))
    cost = (1/2)*(prediction_sparse_matrix - ratings_sparse_matrix).power(2).sum() + user_regularization + song_regularization
    return cost

#### Computing gradients

To learn the parameters, we have to minimize the cost function $ J (x^{(1)},...,x^{(n_{m})}, c^{(1)},...,c^{(n_{u})}) $ given above with respect to both $ x^{(i)} $ and $ c^{(i)} $

The derivative for the cost function with respect to $ x^{(i)} $ is
$$\frac{\mathrm{d}J}{\mathrm{d}x_{k}^{(i)}}  =  \sum_{j:r(i,j)=1} ((c^{(j)})^{T}x^{(i)} - y^{(i,j)})c_{k}^{(j)}  +  \lambda x_{k}^{(i)} $$

The derivative for the cost function with respect to $ c^{(i)} $ is
$$\frac{\mathrm{d}J}{\mathrm{d}c_{k}^{(i)}}  =   \sum_{i:r(i,j)=1} ((c^{(j)})^{T}x^{(i)} - y^{(i,j)})x_{k}^{(i)}  +  \lambda c_{k}^{(j)} $$

In [ ]:
def compute_gradients(user_features, song_features, predicted_ratings_sparse, actual_ratings_sparse, lambd=0.1):
    prediction_offset_sparse = predicted_ratings_sparse - actual_ratings_sparse
    d_users = csr_matrix.dot(prediction_offset_sparse, song_features) + lambd*user_features
    d_songs = csr_matrix.dot(csr_matrix.transpose(prediction_offset_sparse), user_features) + lambd*song_features
    return d_users, d_songs

#### Updating parameters by using computed gradients

$$ x_{k}^{(i)}  =  x_{k}^{(i)}  -  \alpha\frac{\mathrm{d}J}{\mathrm{d}x_{k}^{(i)}} $$
$$ c_{k}^{(i)}  =  c_{k}^{(i)}  -  \alpha\frac{\mathrm{d}J}{\mathrm{d}c_{k}^{(i)}} $$

Where, $\alpha$ is the learning rate

In [ ]:
def update_features(user_features, song_features, d_users, d_songs, learning_rate=0.01):
    user_features = user_features - learning_rate*d_users
    song_features = song_features - learning_rate*d_songs
    return user_features, song_features

#### Comupte error

This is similar to the first part of the cost function. It is the Square root of sum of squared errors for each prediction. This is a useful evaluation metric

$$ \sqrt{\sum_{(i,j):r(i,j)=1} ((c^{(j)})^{T}x^{(i)} - y^{(i,j)})^{2}} $$

In [ ]:
def compute_error(true_ratings, predicted_ratings):
    return np.sqrt((predicted_ratings - true_ratings).power(2).sum())

#### Reading the dataset

For this algorithm, we will use the [songs dataset from kaggle](https://www.kaggle.com/rymnikski/dataset-for-collaborative-filters)  
To split the train and test data, we randomly select one observation for each user as a test observation and use the square root of sum of squared errors to determine the goodness of the model.  

In [ ]:
# Reading data
data = pd.read_csv('data/songsDataset.csv')
data['songIDx'] = data['songID'].astype('category')
data['songIDx'] = data['songIDx'].cat.codes
data['pkey'] = np.arange(data.shape[0])

# Selecting one rating by each user to test properly
test_data = data.groupby(['userID'], as_index=False).apply(lambda x : x.loc[np.random.choice(x.index, replace=False)])
train_data = data.loc[~data['pkey'].isin(test_data['pkey']), :]

#### Declaring some constants and verifying the dataset.

we will use,  
$n\_f$ to denote number of features.  
$n\_u$ to denote number of users.  
$n\_m$ to denote number of songs.  


In [ ]:
n_f = 5
n_u = data['userID'].unique().shape[0]
n_m = data['songID'].unique().shape[0]
num_epochs = 51
lambd = 1
learning_rate = 0.01

print('Total number of observations in training data is {}'.format(train_data.shape[0]))
print('Total number of observations in testing data is {}'.format(test_data.shape[0]))
print('Number of features for both products and users is {}'.format(n_f))
print('Number of unique users in the data is {}'.format(n_u))
print('Number of unique products in the data is {}'.format(n_m))

#### Creating the ratings sparse matrix

We use scipy sparse matrix library to construct actual ratings sparse matrix for both the training and the testing dataset

In [ ]:
# Constructing the train_ratings sparse matrix
train_users = train_data['userID']
train_songs = train_data['songIDx']
train_ratings = csr_matrix((train_data['rating'], (train_users, train_songs)))

# Constructing the test_ratings sparse matrix
test_users = test_data['userID']
test_songs = test_data['songIDx']
test_ratings = csr_matrix((test_data['rating'], (test_users, test_songs)))

In [ ]:
# Training
user_features, song_features = initialize_features(n_u, n_m, n_f)
for eph in range(num_epochs):
    predicted_ratings_sparse = predict_ratings(user_features, song_features, train_users, train_songs)
    print(predicted_ratings_sparse[0,6706])
    if(eph % 2 == 0):
        print("Cost at epoch {} is {}".format(eph, compute_cost(user_features, song_features, predicted_ratings_sparse, train_ratings, lambd)))
    if(eph % 5 == 0):
        test_predicts = predict_ratings(user_features, song_features, test_users, test_songs)
        print("Mean squared error on test data after epoch {} is {}".format(eph, compute_error(test_ratings, test_predicts)))
        print("Mean squared error on train data after epoch {} is {}".format(eph, compute_error(train_ratings, predicted_ratings_sparse)))
    d_users, d_songs = compute_gradients(user_features, song_features, predicted_ratings_sparse, train_ratings, lambd)
    user_features, song_features = update_features(user_features, song_features, d_users, d_songs, learning_rate)